In [ ]:
from huggingface_hub import login
login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch
device = "cuda" # the device to load the model onto

model_name = "Qwen/Qwen1.5-MoE-A2.7B"

# 4-bit QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 # QLoRA = 4-bit base model
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",         # QLoRA uses nf4
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
model.eval()
inputs = tokenizer("Hello!", return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(out[0], skip_special_tokens=True))

Hello! I'm a 20 year old girl from the Netherlands. I'm a student and I'm


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from peft import IA3Config, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

config = LoraConfig(r = 4,
                    lora_alpha=4,
                    target_modules = ["gate", "q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
                    lora_dropout=0.1
                    )

lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()

trainable params: 63,706,752 || all params: 14,379,490,944 || trainable%: 0.4430


In [7]:
from datasets import load_dataset

dataset = load_dataset("Na0s/sft-ready-Text-Generation-Augmented-Data", split="train")

README.md:   0%|          | 0.00/344 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

data/train-00000-of-00022.parquet:   0%|          | 0.00/332M [00:00<?, ?B/s]

data/train-00001-of-00022.parquet:   0%|          | 0.00/323M [00:00<?, ?B/s]

...

data/train-00019-of-00022.parquet:   0%|          | 0.00/119M [00:00<?, ?B/s]

data/train-00020-of-00022.parquet:   0%|          | 0.00/98.5M [00:00<?, ?B/s]

data/train-00021-of-00022.parquet:   0%|          | 0.00/109M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/22 [00:00<?, ?it/s]

In [9]:
print(dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 7667416
})


In [26]:
def tokenize(batch):
    texts = []
    for prompt, completion in zip(batch["prompt"], batch["completion"]):
        text = tokenizer.apply_chat_template(
            [
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": completion},
            ],
            tokenize=False,
            add_generation_prompt=False,
        )

        if isinstance(text, list):
            text = "".join(text)

        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding=False,
        max_length=2048
    )

batch = dataset.select(range(3))
tokens = tokenize(batch)
tokenizer.decode(tokens["input_ids"][0])


'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n0.002 = 1000 \n1 = x?<|im_end|>\n<|im_start|>assistant\nTo find the value of x, we can set up a proportion using the given information:\n\n0.002/1000 = 1/x\n\nTo solve for x, we can cross multiply:\n\n0.002 * x = 1000 * 1\n\n0.002x = 1000\n\nDividing both sides by 0.002:\n\nx = 1000 / 0.002\n\nx = 500,000\n\nTherefore, 1 is equal to 500,000 in this proportion.<|im_end|>\n'

In [27]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    num_proc=75,
    remove_columns=["prompt", "completion"],
)

print(tokenized_dataset[0])

Map (num_proc=75):   0%|          | 0/7667416 [00:00<?, ? examples/s]

{'input_ids': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 15, 13, 15, 15, 17, 284, 220, 16, 15, 15, 15, 715, 16, 284, 856, 30, 151645, 198, 151644, 77091, 198, 1249, 1477, 279, 897, 315, 856, 11, 582, 646, 738, 705, 264, 21117, 1667, 279, 2661, 1995, 1447, 15, 13, 15, 15, 17, 14, 16, 15, 15, 15, 284, 220, 16, 10776, 271, 1249, 11625, 369, 856, 11, 582, 646, 5312, 30270, 1447, 15, 13, 15, 15, 17, 353, 856, 284, 220, 16, 15, 15, 15, 353, 220, 16, 271, 15, 13, 15, 15, 17, 87, 284, 220, 16, 15, 15, 15, 271, 12509, 6577, 2176, 11067, 553, 220, 15, 13, 15, 15, 17, 1447, 87, 284, 220, 16, 15, 15, 15, 608, 220, 15, 13, 15, 15, 17, 271, 87, 284, 220, 20, 15, 15, 11, 15, 15, 15, 271, 54815, 11, 220, 16, 374, 6144, 311, 220, 20, 15, 15, 11, 15, 15, 15, 304, 419, 21117, 13, 151645, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [29]:
print(tokenized_dataset[0:5])
tokenized_dataset.save_to_disk("./tokenized_dataset")

{'input_ids': [[151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 15, 13, 15, 15, 17, 284, 220, 16, 15, 15, 15, 715, 16, 284, 856, 30, 151645, 198, 151644, 77091, 198, 1249, 1477, 279, 897, 315, 856, 11, 582, 646, 738, 705, 264, 21117, 1667, 279, 2661, 1995, 1447, 15, 13, 15, 15, 17, 14, 16, 15, 15, 15, 284, 220, 16, 10776, 271, 1249, 11625, 369, 856, 11, 582, 646, 5312, 30270, 1447, 15, 13, 15, 15, 17, 353, 856, 284, 220, 16, 15, 15, 15, 353, 220, 16, 271, 15, 13, 15, 15, 17, 87, 284, 220, 16, 15, 15, 15, 271, 12509, 6577, 2176, 11067, 553, 220, 15, 13, 15, 15, 17, 1447, 87, 284, 220, 16, 15, 15, 15, 608, 220, 15, 13, 15, 15, 17, 271, 87, 284, 220, 20, 15, 15, 11, 15, 15, 15, 271, 54815, 11, 220, 16, 374, 6144, 311, 220, 20, 15, 15, 11, 15, 15, 15, 304, 419, 21117, 13, 151645, 198], [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 198, 15, 25, 15, 15, 198, 12555, 594, 2087, 389, 7598, 432, 594, 19122, 62, 16, 504, 198, 15, 25, 1

Saving the dataset (0/26 shards):   0%|          | 0/7667416 [00:00<?, ? examples/s]

In [30]:

tokenizer.decode(tokenized_dataset["input_ids"][0])

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n0.002 = 1000 \n1 = x?<|im_end|>\n<|im_start|>assistant\nTo find the value of x, we can set up a proportion using the given information:\n\n0.002/1000 = 1/x\n\nTo solve for x, we can cross multiply:\n\n0.002 * x = 1000 * 1\n\n0.002x = 1000\n\nDividing both sides by 0.002:\n\nx = 1000 / 0.002\n\nx = 500,000\n\nTherefore, 1 is equal to 500,000 in this proportion.<|im_end|>\n'

In [20]:
print(dataset[0])

{'prompt': '0.002 = 1000 \n1 = x?', 'completion': 'To find the value of x, we can set up a proportion using the given information:\n\n0.002/1000 = 1/x\n\nTo solve for x, we can cross multiply:\n\n0.002 * x = 1000 * 1\n\n0.002x = 1000\n\nDividing both sides by 0.002:\n\nx = 1000 / 0.002\n\nx = 500,000\n\nTherefore, 1 is equal to 500,000 in this proportion.'}


In [15]:
from trl import SFTConfig
import logging
logging.getLogger("trl.trainer.sft_trainer").setLevel(logging.ERROR)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-MoE-A2.7B", use_fast=True)

trainer = SFTTrainer(
    model = lora_model,
    train_dataset = dataset,
    processing_class = tokenizer,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 16,
        packing = True,
        group_by_length = True,
        warmup_steps = 5,
        bf16 = True,
        max_steps=10000,
        learning_rate = 2e-4,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        eval_strategy="no",
        do_eval=False,
        output_dir = "./outputs",
        push_to_hub=True,
        remove_unused_columns=False,
    )
)

torch.cuda.empty_cache()

trainer.train()

trainer.model.save_pretrained("Qwen1.5_MoE_lora_model")

Tokenizing train dataset:   0%|          | 0/7667416 [00:00<?, ? examples/s]

KeyboardInterrupt: 